In [ ]:
import os
import cv2
import json
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from copy import deepcopy
from pprint import pprint
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

### Example Annotation

In [ ]:
annotation_path = "/Users/shashanks./college/rrc/dataset/mpii/annot/train.json"
dataset_path = "/Users/shashanks./college/rrc/dataset/mpii/images"

with open(annotation_path, 'r') as f:
    annotation_json = json.load(f)

pprint(annotation_json[0])

### MPII Flipped pairs

In [ ]:
flip_pairs = [
    [0, 5], # Right and Left Ankle
    [1, 4], # Right and Left Knee
    [2, 3], # Right and Left Hip
    [10, 15], # Right and Left Wrist
    [11, 14], # Right and Left Elbow
    [12, 13] # Right and Left Shoulder 
]

### Display bbox image

In [ ]:
idx = 4356
img_path = os.path.join(dataset_path, annotation_json[idx]['image'])
center = annotation_json[idx]['center']
scale = annotation_json[idx]['scale']
joints = np.array(annotation_json[idx]['joints'])
visible_joints = np.array(annotation_json[idx]['joints_vis'])

# Adjust center/scale slightly to avoid cropping limbs
if center[0] != -1:
    center[1] = center[1] + 15 * scale
    scale = scale * 1.25

In [ ]:
center

In [ ]:
scale = scale * 200
x, y = int(center[0] - scale / 2.0), int(center[1] - scale / 2.0)
w, h = int(scale), int(scale)
x, y

In [ ]:
w, h

In [ ]:
def disp_bbox_image(img_path):
    img = cv2.imread(img_path)[:,:,::-1] #OpenCV uses BGR channels

    fig = plt.figure(figsize=(5, 5))
    ax1 = fig.add_subplot(1, 1, 1)

    ax1.imshow(img)

    # Create a Rectangle patch
    rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    plt.scatter([x], [y])

    # Add the patch to the Axes
    ax1.add_patch(rect)
    plt.show()
    
    return img

In [ ]:
img = disp_bbox_image(img_path)

In [ ]:
img.shape

### Crop Image

In [ ]:
class Crop(object):
    def __init__(self, size):
        self.size = size
    
    def __call__(self, sample):
        img, keypoints, visible_keypoints, scale = sample["image"], sample["keypoints"], sample["visible_keypoints"], sample["scale"]
        x, y = int(center[0] - scale / 2.0), int(center[1] - scale / 2.0)
        # Row, cols are interchanged in image and numpy arrays
        row, col = y, x
        cropped_image = img[row:row + h, col:col + w]
        
        for i in range(keypoints.shape[0]):
            if keypoints[i, 0] < x or keypoints[i, 0] > x + w or keypoints[i, 1] < y or keypoints[i, 1] > y + h:
                visible_keypoints[i] = 0

In [ ]:
w, h

In [ ]:
# row, col = y, x
# cropped_image = deepcopy(img[row:row + h, col:col + w])
# plt.imshow(cropped_image)

row, col = y, x
cropped_image = img[row:row + h, col:col + w]
plt.imshow(cropped_image)

In [ ]:
cropped_image.shape

### Display Keypoints image

In [ ]:
def disp_keypoints_image(img, keypoints):    
    fig = plt.figure(figsize=(7, 7))
    ax1 = fig.add_subplot(1, 1, 1)
    keypoints_num = list(range(len(keypoints)))
    ax1.imshow(img)
    ax1.scatter(keypoints[:, 0], keypoints[:, 1])
    
    for i, txt in enumerate(keypoints_num):
        ax1.annotate(txt, (keypoints[i, 0], keypoints[i, 1]), c='w')
    plt.show()

In [ ]:
disp_keypoints_image(img, joints)

### Generate samples

In [ ]:
sample = {
    "image": img, 
    "keypoints": joints,
    "visible_keypoints": visible_joints
}

### Horizontal Flip Transform

In [ ]:
class RandomHorizontalFlip(object):
    def __init__(self, prob=0.5):
        assert prob >= 0 and prob <= 1, "Invalid probability"
        self.prob = prob

    def __call__(self, sample):
        img, keypoints, visible_keypoints = sample["image"], sample["keypoints"], sample["visible_keypoints"]
        img_shape = np.array(img.shape[:2])
        # Get the centerpoints, we flip rows and columns as we are dealing with matrices
        img_center = img_shape[::-1] / 2
        
        if np.random.rand() < self.prob:
            img =  img[:, ::-1, :] # Flip the image
            keypoints[:, 0] += 2 * (img_center[0] - keypoints[:, 0])
            
            for pair in flip_pairs:
                idx_1, idx_2 = pair
                keypoints[idx_1, :], keypoints[idx_2, :] = keypoints[idx_2, :], keypoints[idx_1, :].copy()
                visible_keypoints[idx_1], visible_keypoints[idx_2] = visible_keypoints[idx_2], visible_keypoints[idx_1].copy()

            updated_sample = {
                "image": img, 
                "keypoints": keypoints, 
                "visible_keypoints": visible_keypoints
            }
            return updated_sample
        return sample

In [ ]:
r = RandomHorizontalFlip()
updated_sample = r(sample)

In [ ]:
disp_keypoints_image(updated_sample["image"], updated_sample["keypoints"])

### Scale Transform

In [ ]:
sample = {
    "image": img, 
    "keypoints": joints,
    "visible_keypoints": visible_joints
}

In [ ]:
class RandomScale(object):
    """
    Parameters
    ----------
    scale: float or tuple(float)
        if **float**, the image is scaled by a factor drawn 
        randomly from a range (1 - `scale` , 1 + `scale`). If **tuple**,
        the `scale` is drawn randomly from values specified by the 
        tuple
    diff: bool
        if **true**, the x & y dimensions are scaled equally, else they 
        are scaled separately.    
    """
    
    def __init__(self, scale, diff):
        self.scale = scale
        self.diff = diff
        if type(self.scale) != tuple:
            self.scale = max((-1, -self.scale), self.scale)
    
    def __call__(self, sample):
        img, keypoints, visible_keypoints = sample["image"], sample["keypoints"], sample["visible_keypoints"]
        img_shape = img.shape
        if self.diff:
            scale_x = np.random.uniform(*self.scale)
            scale_y = np.random.uniform(*self.scale)
        else:
            scale_x = np.random.uniform(*self.scale)
            scale_y = scale_x
        
        rescale_x = 1 + scale_x
        rescale_y = 1 + scale_y
        
        updated_img = cv2.resize(img, fx=rescale_x, fy=rescale_y)
        updated_keypoints = deepcopy(keypoints)
        updated_keypoints[:, 0] = updated_keypoints[:, 0] * rescale_x
        updated_keypoints[:, 1] = updated_keypoints[:, 1] * rescale_y
        
        canvas = np.zeros(img_shape, dtype=np.uint8)
        # X and Y axes are flipped in arrays and images
        x_lim = np.min(img_shape[1], updated_img.shape[1])
        y_lim = np.min(img_shape[0], updated_img.shape[0])

In [ ]:
img, keypoints, visible_keypoints = updated_sample["image"], updated_sample["keypoints"], updated_sample["visible_keypoints"]

In [ ]:
keypoints.shape 

In [ ]:
keypoints1 = deepcopy(keypoints)
keypoints1[:, 0] = keypoints1[:, 0] * 1.35
keypoints1[:, 1] = keypoints1[:, 1] * 2.25

# keypoints1[:, 0] = keypoints1[:, 0] * 2.25
# keypoints1[:, 1] = keypoints1[:, 1] * 1.35

In [ ]:
keypoints1[7]

In [ ]:
updated_img.shape

In [ ]:
updated_img = cv2.resize(img, None, fx=1.35, fy=2.25)

plt.scatter(keypoints1[7, 0], keypoints1[7, 1])

plt.imshow(updated_img)
# plt.imshow(img)

In [ ]:
class RandomScale(object):
    """
    Parameters
    ----------
    scale: float or tuple(float)
        if **float**, the image is scaled by a factor drawn 
        randomly from a range (1 - `scale` , 1 + `scale`). If **tuple**,
        the `scale` is drawn randomly from values specified by the 
        tuple
    diff: bool
        if **true**, the x & y dimensions are scaled equally, else they 
        are scaled separately.
    """
    def __init__(self, scale=0.25, diff=False):
        self.scale = scale
        if type(scale) == tuple:
            assert len(self.scale) == 2, "Invalid range"
            assert self.scale[0] > -1, "Scale factor can't be less than -1"
            assert self.scale[1] > -1, "Scale factor can't be less than -1"
        else:
            assert self.scale > 0, "Please input a positive float"
            self.scale = (max(-1, -self.scale), self.scale)
        self.diff = diff
    
    def __call__(self, sample):
        img, keypoints, visible_keypoints = sample["image"], sample["keypoints"], sample["visible_keypoints"]
        if self.diff:
            scale_x = np.random.uniform(*self.scale)
            scale_y = np.random.uniform(*self.scale)
        else:
            scale_x = np.random.uniform(*self.scale)
            scale_y = scale_x
        
        resize_scale_x = 1 + scale_x
        resize_scale_y = 1 + scale_y
        
        img = cv2.resize(img, None, fx=resize_scale_x, fy=resize_scale_y)
        keypoints[:, 0] *= resize_scale_x
        keypoints[:, 1] *= resize_scale_y
        
        updated_sample = {
            "image": img,
            "keypoints": keypoints,
            "visible_keypoints": visible_keypoints
        }
        return updated_sample

In [ ]:
s = RandomScale((0.65, 1.30))
updated_sample = s(sample)

In [ ]:
disp_keypoints_image(updated_sample["image"], updated_sample["keypoints"])